# HW4: QA Agent

## Dependencies and LLM Backbone

In [1]:
# !pip install langchain==1.0.5
# !pip install langchain-core
# !pip install langchain-community
# !pip install faiss-cpu
# !pip install kagglehub

In [2]:
# This is the list of countries we are using (with their official languages)
# Feel free to use it in your code
list_of_countries = {}
with open("countries_with_languages.tsv", "r"  ) as f:
    for line in f.readlines():
        country, langs = line.strip().split("\t")
        list_of_countries[country] = langs.split(",")

### Choice 1: OpenAI API

The notebook's implementation is based on this.
Feel free to change the model, and please keep track of your usage on the "Usage" page on [LiteLLM API webpage](https://ai-gateway.andrew.cmu.edu/ui/).

In [3]:
# !pip install langchain-openai

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import getpass, os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")
openai_model_id = "gpt-4o-mini-2024-07-18"
openai_embmodel_id = "azure/text-embedding-3-small"

llm = ChatOpenAI(
    model=openai_model_id,
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://ai-gateway.andrew.cmu.edu/"
)
embeddings = OpenAIEmbeddings(
    model=openai_embmodel_id,
    api_key=os.environ['OPENAI_API_KEY'],
    base_url='https://ai-gateway.andrew.cmu.edu/'
)

### Choice 2: Hugging Face Models

You may also use Hugging Face models without API credits if you have available GPU resource. You might have to the change prompt templates according to your model choice.

In [5]:
# !pip install langchain-huggingface text-generation transformers google-search-results 
# !pip install numexpr langchainhub sentencepiece jinja2 bitsandbytes accelerate

In [6]:
# import getpass, os, torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline, HuggingFaceEmbeddings

# os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")
# hgf_model_id = "Qwen/Qwen3-0.6B"
# hgf_embmodel_id = "sentence-transformers/all-mpnet-base-v2"

# # Select device and dtype
# device = "cuda" if torch.cuda.is_available() else "cpu"
# dtype = torch.float16 if device == "cuda" else torch.float32

# # Build HF generation pipeline on GPU when available
# tokenizer = AutoTokenizer.from_pretrained(hgf_model_id, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     hgf_model_id,
#     torch_dtype=dtype,
#     device_map="auto",
#     trust_remote_code=True,
# )

# gen_pipe = pipeline(
#     task="text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     device_map="auto",
#     torch_dtype=dtype,
# )

# hgf_model = HuggingFacePipeline(pipeline=gen_pipe)
# hgf_llm = ChatHuggingFace(llm=hgf_model)

# # Place sentence-transformers embeddings on GPU when available
# hgf_embeddings = HuggingFaceEmbeddings(
#     model_name=hgf_embmodel_id,
#     model_kwargs={"device": device}
# )

## Handling different type of questions

Implement the answer formatting and extraction for each question type. You may change the prompt to fit your processing function.

In [7]:
from langchain.agents import create_agent

### 🗺️Global Trekker

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool

@tool
def web_search(query: str) -> str:
    """
    Searches the web for information about locations, landmarks, and geographic features.
    Useful for identifying cities and countries from descriptive clues.
    """
    search = DuckDuckGoSearchRun()
    try:
        results = search.run(query)
        return results
    except Exception as e:
        return f"Search error: {str(e)}"

ValueError: Function must have a docstring if description not provided.

In [ ]:
global_trekker_messages = [
    {"role": "system", "content": "You are an expert in world geography and cultural geography. Given a descriptive paragraph, infer the most likely country and city with high precision."},
    {"role": "user", "content": """
Read the paragraph and extract:
- Country: use the full official country name (e.g., "United States", not "USA").
- City: a specific city if clearly indicated by clues (landmarks, neighborhoods, transit lines, local foods, dialects). If not identifiable, write "Unknown".

Strict output format (no explanations):
[Country], [City]

Guidelines:
- Prefer unique cues (local transit names, street names, postal formats, phone codes, currency, cuisine, sports clubs).
- If multiple cities fit, pick the single most likely one.
- If the paragraph is generic or only country-level, return "Unknown" for city.
- Do not include any extra text besides the bracketed pair.

Examples:
Input: "We walked along the Arno past the Ponte Vecchio before climbing to Piazzale Michelangelo."
Output: [Italy], [Florence]

Input: "A red double-decker bus passed by the Thames near Westminster Abbey and Big Ben."
Output: [United Kingdom], [London]

Input: "We grabbed Primanti's near the confluence of the Allegheny and Monongahela."
Output: [United States], [Pittsburgh]

Input: "I toured the Old Town and the Royal Mile before seeing the castle on the hill."
Output: [United Kingdom], [Edinburgh]

Input: "We enjoyed maple syrup and poutine while skating on a canal in winter."
Output: [Canada], [Ottawa]
"""},
]
global_trekker = create_agent(model=llm, tools=[web_search])

In [ ]:
def extract_global_trekker_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and city from the response
    # return country, city
    import re
    
    # Look for pattern [country], [city] or variations
    # Try to find content within brackets first
    bracket_pattern = r'\[([^\]]+)\]\s*,\s*\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        city = match.group(2).strip()
        return country, city
    
    # Fallback: look for two items separated by comma
    # Often LLMs will say something like "United States, Pittsburgh"
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Look for comma-separated values
        if ',' in line:
            parts = line.split(',')
            if len(parts) >= 2:
                country = parts[0].strip().strip('[]"\'')
                city = parts[1].strip().strip('[]"\'')
                # Clean up common prefixes
                for prefix in ['The answer is', 'Answer:', 'Location:', 'Country:', 'City:']:
                    country = country.replace(prefix, '').strip()
                    city = city.replace(prefix, '').strip()
                return country, city
    
    # Last resort: return empty strings
    return "", ""

### 🍽️Culinary Detective

In [ ]:
import kagglehub
import pandas as pd

def gather_recipe_data(kaggledataset: str) -> list[str]:
    dataset_path = kagglehub.dataset_download(kaggledataset)
    df = pd.read_csv(f"{dataset_path}/Receipes from around the world.csv", encoding='latin-1')
    
    # Process the dataframe to list of text entries for retrieval
    # Format each recipe as structured text for better retrieval
    recipes = []
    for _, row in df.iterrows():
        # Create a readable text representation of each recipe
        recipe_parts = []
        for col in df.columns:
            value = row[col]
            # Skip NaN values and format nicely
            if pd.notna(value) and str(value).strip():
                recipe_parts.append(f"{col}: {value}")
        
        # Join all parts into a single text entry
        recipe_text = ". ".join(recipe_parts)
        recipes.append(recipe_text)
    
    return recipes

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.tools import tool

recipes = gather_recipe_data("prajwaldongre/collection-of-recipes-around-the-world")
docs = [Document(page_content=recipe) for recipe in recipes]
vector = FAISS.from_documents(docs, embeddings)
retriever = vector.as_retriever(search_kwargs={"k": 2})

## RAG Output Before Entering the LLM
This piece of code prints out what the RAG system is outputing before feeding it into the LLM. I wrote it personally to examine what the RAG is outputting, to see if the RAG is misleading or not. 

In [ ]:
# Show the RAG retrieval output for question 93
q = questions[103]

if q["type"] == "CulinaryDetective":
    # Build the query as it would be sent to the retriever
    query_text = f"Ingredients: {q['ingredient']}. Description: {q['description']}"
    
    print(f"Question Type: {q['type']}")
    print(f"\nQuery sent to retriever:")
    print(f"{query_text}")
    print("\n" + "="*80)
    print("Retrieved Documents (RAG Output):")
    print("="*80)
    
    # Get the retrieval results
    retrieved_docs = retriever.invoke(query_text)
    
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\n[Document {i}]")
        print(doc.page_content)
        print("-"*80)
else:
    print(f"Question 93 is of type '{q['type']}', not 'CulinaryDetective'.")
    print("The RAG system is only used for CulinaryDetective questions.")

Question Type: CulinaryDetective

Query sent to retriever:
Ingredients: Salted cod, onions, thinly shredded potatoes, eggs, olive oil, parsley, black olives. Description: Sautéed cod with onions and shredded potatoes, mixed with eggs, savory, slightly salty, golden and textured

Retrieved Documents (RAG Output):

[Document 1]
recipe_name: Spanish Tortilla Española (Spanish Omelette). cuisine: Spanish. ingredients: ['Potatoes', 'Onions', 'Eggs', 'Olive oil', 'Salt', 'Pepper']. cooking_time_minutes: 40. prep_time_minutes: 15. servings: 6.0. calories_per_serving: 200.0. dietary_restrictions: ['vegetarian', 'gluten-free', 'dairy-free']
--------------------------------------------------------------------------------

[Document 2]
recipe_name: Swedish Potatoes Delight. cuisine: Swedish. ingredients: ['potatoes', 'lingonberries', 'salmon', 'cream']. cooking_time_minutes: 51. prep_time_minutes: 36. servings: 4.0. calories_per_serving: 372.0. dietary_restrictions: ['nan']
----------------------

In [ ]:
@tool
def retrieve_recipes(query: str):
  """
  Retrieves recipes based on a search query.
  """
  return retriever.invoke(query)

culinary_detective_messages = [
    {"role": "system", "content": """
You are an expert culinary anthropologist. From ingredients and a short description, identify the country and the SPECIFIC region within that country where the dish is most associated.

You can consult a retrieval tool bound to this agent (retrieve_culinary_context). Use it when helpful; otherwise reason from your knowledge of ingredients, techniques, and named dishes.
"""},
    {"role": "human", "content": """
Task:
Return ONLY a single line in the exact format: [Country], [Region]
- Country: full official name (e.g., "United States", not "USA").
- Region: a specific intra-country region (e.g., North, South, East, West, Central, Northeast, etc.).
- Use "All" only if the dish is truly nationwide.
- If no region is identifiable, write "Unknown".
- Do NOT include any explanation before or after the bracketed answer.

Cues to consider:
- Ingredients (grains, staple flours, spice blends), cooking methods, named dishes, iconic sides.
- Example mappings:
  * Brazil: pão de queijo / tapioca flour / queijo minas → [Brazil], [South]
  * Japan: sushi / rice vinegar / tempura → [Japan], [All]
  * India: dosa / idli / coconut → [India], [South]; naan / paneer / tandoor → [India], [North]
  * Ethiopia: teff / injera / wat / berbere → [Ethiopia], [Unknown]
  * China: dim sum → [China], [South]; hot pot (Sichuan/Chongqing-style) → [China], [West]
  * Thailand: khao soi → [Thailand], [North]; som tam → [Thailand], [Northeast]

Few-shot examples:
Input: "Fermented teff flatbread served with spicy stews (wat) and berbere."
Output: [Ethiopia], [Unknown]

Input: "Cheese bread made with tapioca starch, typical with churrasco in the south."
Output: [Brazil], [South]

Input: "Batter of rice and urad dal, steamed into soft cakes, served with coconut chutney."
Output: [India], [South]

Input: "Assorted small bites with tea in bamboo steamers, hallmark of Cantonese cuisine."
Output: [China], [South]

Now produce ONLY the answer for the current input as [Country], [Region].
"""},
]
culinary_detective = create_agent(model=llm, tools=[retrieve_recipes])

In [ ]:
def extract_culinary_detective_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and region from the response
    # return country, region
    import re
    
    # Look for pattern [country], [region] or variations
    bracket_pattern = r'\[([^\]]+)\]\s*,\s*\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        region = match.group(2).strip()
        return country, region
    
    # Fallback: look for two items separated by comma
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Look for comma-separated values
        if ',' in line:
            parts = line.split(',')
            if len(parts) >= 2:
                country = parts[0].strip().strip('[]"\'')
                region = parts[1].strip().strip('[]"\'')
                # Clean up common prefixes
                for prefix in ['The answer is', 'Answer:', 'Country:', 'Region:']:
                    country = country.replace(prefix, '').strip()
                    region = region.replace(prefix, '').strip()
                return country, region
    
    # Last resort: return empty strings
    return "", ""

### 👄Lingua Locale

In [ ]:
lingua_locale_messages = [
    {"role": "system", "content": """
You are an expert in languages, scripts, orthography, and regional vocabulary. Determine which country's website or text a sentence most likely comes from.
"""},
    {"role": "user", "content": """
Return ONLY a single line: [Country]
- Use the full official country name (e.g., "United States", not "USA").
- No explanations. Do not output anything else.

Heuristics:
- English: colour, organise, centre → [United Kingdom]; sidewalk, color, organize → [United States].
- Portuguese: autocarro, factura, telemóvel → [Portugal]; ônibus, nota fiscal, celular → [Brazil].
- Spanish: vos, colectivo, pileta → [Argentina]; ordenador → [Spain]; autobús → [Mexico/Spain], coche (Spain) vs carro (LatAm).
- Chinese: Traditional characters (臺灣、臺北、繁體) → [Taiwan]; Simplified (中国、广州、简体) → [China].
- Cyrillic specifics: дј / ђ / ћ / љ / њ → [Montenegro]; девојка / ђак more typical of [Serbia].
- French: anglicisms + CAD context → [Canada]; métropolitain cues → [France].
- Haitian Creole → [Haiti].

Examples:
Input: "Please, colour is the preferred spelling in our centre."
Output: [United Kingdom]

Input: "Clique para imprimir a fatura no seu telemóvel."
Output: [Portugal]

Input: "Синоћ сам видио дјевојку у Подгорици."
Output: [Montenegro]

Input: "這是臺灣本地的頁面。"
Output: [Taiwan]

Now produce ONLY [Country] for the current sentence.
"""},
]
lingua_locale = create_agent(model=llm, tools=[])

In [ ]:
def extract_lingua_locale_answer(response: str) -> tuple[str, str]:
    # TODO: Extract the country and "none" from the response
    # only the first field is used, the second is a dummy field to make the return type consistent
    # return country, "none"
    import re
    
    # Look for pattern [country] in brackets
    bracket_pattern = r'\[([^\]]+)\]'
    match = re.search(bracket_pattern, response)
    
    if match:
        country = match.group(1).strip()
        return country, "none"
    
    # Fallback: look for country name in response
    lines = response.strip().split('\n')
    for line in lines:
        # Skip lines that are too long (likely explanations)
        if len(line) > 100:
            continue
        # Clean the line
        clean_line = line.strip().strip('[]"\'')
        # Remove common prefixes
        for prefix in ['The answer is', 'Answer:', 'Country:', 'The country is', 'This is from']:
            clean_line = clean_line.replace(prefix, '').strip()
        
        # If we have a short, cleaned line, it's likely the country
        if clean_line and len(clean_line) < 50:
            # Remove any trailing punctuation
            clean_line = clean_line.rstrip('.,;:')
            return clean_line, "none"
    
    # Last resort: return empty string
    return "", "none"

## Answering questions
This part includes how we load the questions and generate the prediction in desired format. 

In [ ]:
def geoguesser(q: dict, print_raw_response=False) -> tuple[str, str]:
    if q["type"] == "GlobalTrekker":
        query = {"role": "user", "content": f"Paragraph: {q["paragraph"]}"}
        messages, agent, extractor = global_trekker_messages, global_trekker, extract_global_trekker_answer
    elif q["type"] == "CulinaryDetective":
        query = {"role": "user", "content": f"Ingredients: {q["ingredient"]}. Description: {q["description"]}"}
        messages, agent, extractor = culinary_detective_messages, culinary_detective, extract_culinary_detective_answer
    else: #q["type"] == "LinguaLocale":
        query = {"role": "user", "content": f"Sentence: {q["sentence"]}"}
        messages, agent, extractor = lingua_locale_messages, lingua_locale, extract_lingua_locale_answer

    response_all = agent.invoke({"messages": messages + [query]})
    response = response_all["messages"][len(messages)+1].content
    if print_raw_response: print(f"{q["type"]}: {response_all}")
    return extractor(response)

In [ ]:
import json

# Here, we load the examples questions. Public/private set will be in the same format
dataset_name = "public.jsonl"
questions = []
with open(dataset_name, "r") as f:
    for line in f.readlines():
        questions.append(json.loads(line))

In [ ]:
# Test run on one question
# You might want to save the raw response for debugging answer formatting/extraction
# If the extracted answer seems off, check the raw response instead of running inference repeatedly
geoguesser(questions[103], print_raw_response=True)

CulinaryDetective: {'messages': [SystemMessage(content='\nYou are an expert culinary anthropologist. From ingredients and a short description, identify the country and the SPECIFIC region within that country where the dish is most associated.\n\nYou can consult a retrieval tool bound to this agent (retrieve_culinary_context). Use it when helpful; otherwise reason from your knowledge of ingredients, techniques, and named dishes.\n', additional_kwargs={}, response_metadata={}, id='e67d15d8-8018-4b92-9ce5-497a84c1db93'), HumanMessage(content='\nTask:\nReturn ONLY a single line in the exact format: [Country], [Region]\n- Country: full official name (e.g., "United States", not "USA").\n- Region: a specific intra-country region (e.g., North, South, East, West, Central, Northeast, etc.).\n- Use "All" only if the dish is truly nationwide.\n- If no region is identifiable, write "Unknown".\n- Do NOT include any explanation before or after the bracketed answer.\n\nCues to consider:\n- Ingredients

('Portugal', 'All')

In [ ]:
# Sample script to generate answers
answers = []
for q in questions:
    country, category = geoguesser(q)
    answers.append(f"{q["type"]}\t{country}\t{category}")

with open("public.txt", "w") as f:
    for answer in answers:
        f.write(answer + "\n")

## Evaluation
This is how we calculate the scores on Gradescope (details subject to change, but the general logic will stay the same).

In [ ]:
def soft_match(answer, expectedAnswer):
    score = 0.0
    if expectedAnswer in answer:
        score = len(expectedAnswer) / len(answer)
    return score

def exact_match(answer, expectedAnswer):
    score = 0.0
    if expectedAnswer == answer:
        score = 1.0
    return score

In [ ]:
answers = []
for q in questions:
    answers.append((q["type"], q["country"], q.get("city", q.get("region", ""))))
with open("public.txt", "r") as f:
    preds = [line.split("\t") for line in f.readlines()]

scores = {"GlobalTrekker": [], "CulinaryDetective": [], "LinguaLocale": []}
for (q_type, exp_country, exp_place), (p_type, pred_country, pred_place) in zip(answers, preds):
    assert q_type == p_type
    country_score = soft_match(pred_country, exp_country)
    category_score = 0.0
    weights = [0.0, 0.0]
    if q_type == "GlobalTrekker":
        #  correct country -> 80%, correct counrry and city -> +20%
        weights = [0.8, 0.2]
        if exp_place != "" and country_score > 0:
            category_score = soft_match(pred_place, exp_place)
    elif q_type == "CulinaryDetective":
        # correct country -> 60%, correct country and region -> +40%
        weights = [0.6, 0.4]
        if exp_place != "All" and country_score > 0:
            category_score = exact_match(pred_place, exp_place)
    else: # LinguaLocale
        # correct country -> 60%, matched official language -> +40%
        weights = [0.6, 0.4]
        if country_score > 0:
            category_score = 1.0
        else: # incorrect country. language match works only if pred_country is a clean answer
            exp_langs = list_of_countries.get(exp_country, [])
            pred_langs = list_of_countries.get(pred_country, [])
            if any(lang in exp_langs for lang in pred_langs):
                category_score = 1.0

    score = weights[0] * country_score + weights[1] * category_score
    scores[q_type].append(score)

for q_type, score_list in scores.items():
    avg_score = sum(score_list) / len(score_list)
    print(f"{q_type} Average Score: {avg_score:.4f}")

GlobalTrekker Average Score: 0.7048
CulinaryDetective Average Score: 0.2945
LinguaLocale Average Score: 0.8796


# Analysis

### Analyzing mismatched answers

In [ ]:
import json

# Load questions from JSONL
with open("public.jsonl", "r") as f:
    questions = [json.loads(line) for line in f.readlines()]

# Load predictions from TXT
with open("public.txt", "r") as f:
    preds = [line.strip().split("\t") for line in f.readlines()]

# Compare and print only mismatches
print("=" * 100)
print("MISMATCHED ANSWERS ONLY")
print("=" * 100)

mismatch_count = 0
# Track statistics by question type
type_stats = {}

for idx, (q, pred) in enumerate(zip(questions, preds)):
    q_type = q["type"]
    pred_type = pred[0]
    pred_country = pred[1] if len(pred) > 1 else ""
    pred_place = pred[2] if len(pred) > 2 else ""
    
    # Initialize stats for this question type if not seen before
    if q_type not in type_stats:
        type_stats[q_type] = {"total": 0, "mismatches": 0}
    type_stats[q_type]["total"] += 1
    
    # Get expected answers based on question type
    exp_country = q["country"]
    if q_type == "GlobalTrekker":
        exp_place = q.get("city", "")
        place_label = "City"
    elif q_type == "CulinaryDetective":
        exp_place = q.get("region", "")
        place_label = "Region"
    else:  # LinguaLocale
        exp_place = "none"
        place_label = "N/A"
    
    # Check correctness
    country_match = exp_country in pred_country if pred_country else False
    if q_type == "GlobalTrekker":
        place_match = (exp_place in pred_place if exp_place and pred_place else exp_place == pred_place)
    elif q_type == "CulinaryDetective":
        place_match = (pred_place == exp_place)
    else:
        place_match = True
    
    # Only print if there's a mismatch
    if not country_match or not place_match:
        mismatch_count += 1
        type_stats[q_type]["mismatches"] += 1
        
        # Print question info
        print(f"\n[Question {idx + 1}] Type: {q_type}")
        print("-" * 100)
        
        # Print question content
        if q_type == "GlobalTrekker":
            print(f"Paragraph: {q['paragraph'][:100]}...")
        elif q_type == "CulinaryDetective":
            print(f"Ingredients: {q['ingredient']}")
            print(f"Description: {q['description'][:100]}..." if len(q['description']) > 100 else f"Description: {q['description']}")
        else:  # LinguaLocale
            print(f"Sentence: {q['sentence'][:100]}..." if len(q['sentence']) > 100 else f"Sentence: {q['sentence']}")
        
        print()
        print(f"Expected Answer:")
        print(f"  Country: {exp_country}")
        if place_label != "N/A":
            print(f"  {place_label}: {exp_place if exp_place else 'Unknown/Empty'}")
        
        print()
        print(f"Generated Answer:")
        print(f"  Country: {pred_country}")
        if place_label != "N/A":
            print(f"  {place_label}: {pred_place}")
        
        print()
        print(f"Match Status:")
        print(f"  Country: {'✓ MATCH' if country_match else '✗ MISMATCH'}")
        if place_label != "N/A" and exp_place:
            print(f"  {place_label}: {'✓ MATCH' if place_match else '✗ MISMATCH'}")
        
        print("=" * 100)

print(f"\nTotal Questions: {len(questions)}")
print(f"Mismatched Answers: {mismatch_count}")
print(f"Accuracy: {(len(questions) - mismatch_count) / len(questions) * 100:.2f}%")

print("\n" + "=" * 100)
print("STATISTICS BY QUESTION TYPE")
print("=" * 100)
for q_type in sorted(type_stats.keys()):
    total = type_stats[q_type]["total"]
    mismatches = type_stats[q_type]["mismatches"]
    accuracy = (total - mismatches) / total * 100 if total > 0 else 0
    print(f"\n{q_type}:")
    print(f"  Total Questions: {total}")
    print(f"  Mismatched Answers: {mismatches}")
    print(f"  Accuracy: {accuracy:.2f}%")

MISMATCHED ANSWERS ONLY

[Question 1] Type: GlobalTrekker
----------------------------------------------------------------------------------------------------
Paragraph: Uluru, also known as Ayers Rock, is a large sandstone monolith that is sacred to the Pitjantjatjara,...

Expected Answer:
  Country: Australia
  City: Unknown/Empty

Generated Answer:
  Country: Australia
  City: Unknown

Match Status:
  Country: ✓ MATCH

[Question 3] Type: GlobalTrekker
----------------------------------------------------------------------------------------------------
Paragraph: Paradise Island is a vacation destination known for being home to the large Atlantis resort and casi...

Expected Answer:
  Country: Bahamas
  City: Nassau

Generated Answer:
  Country: Bahamas
  City: Unknown

Match Status:
  Country: ✓ MATCH
  City: ✗ MISMATCH

[Question 4] Type: GlobalTrekker
----------------------------------------------------------------------------------------------------
Paragraph: Tomorrowland is an a

In [ ]:
import json
import string
from collections import defaultdict

PUBLIC_TXT = "public.txt"
PUBLIC_JSONL = "public.jsonl"

def normalize(s: str) -> str:
    """Normalize text for fair comparison: strip whitespace, lower-case, remove trailing punctuation like commas/periods."""
    if s is None:
        return ""
    s = s.strip()
    # Remove common trailing punctuation (commas, periods) and surrounding whitespace again
    s = s.rstrip(".,;:")
    return s.casefold()

# -------------------------
# Load generated answers
# Expect tab-separated: TYPE \t Country \t City
# -------------------------
generated = []
with open(PUBLIC_TXT, "r", encoding="utf-8") as f:
    for lineno, raw in enumerate(f, start=1):
        line = raw.rstrip("\n")
        if not line:
            # keep a placeholder for blank lines to preserve indexing
            generated.append(None)
            continue
        parts = line.split("\t")
        # if file uses multiple consecutive whitespace, try splitting by any whitespace as fallback
        if len(parts) != 3:
            parts = line.split()
        if len(parts) == 3:
            qtype, country, city = parts
            generated.append({"type": qtype, "country": country, "city": city})
        else:
            # unparseable line -> store None and print debug
            print(f"Warning: could not parse generated line {lineno!s}: {line!r}")
            generated.append(None)

# -------------------------
# Load expected answers (JSONL)
# We expect each JSON object to have keys at least: "type", "country", "city", "paragraph" (paragraph optional)
# -------------------------
expected = []
with open(PUBLIC_JSONL, "r", encoding="utf-8") as f:
    for lineno, raw in enumerate(f, start=1):
        line = raw.strip()
        if not line:
            # skip blank lines in jsonl
            continue
        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            print(f"Warning: could not parse JSON on line {lineno} of {PUBLIC_JSONL}: {line[:150]!r}")
            continue
        expected.append(obj)

# -------------------------
# If lengths differ, print a notice (this is important for debugging)
# -------------------------
len_expected = len(expected)
len_generated = len(generated)
if len_expected != len_generated:
    print("\n" + "="*80)
    print("NOTICE: file line-count mismatch")
    print(f"  public.jsonl entries (expected): {len_expected}")
    print(f"  public.txt   entries (generated): {len_generated}")
    print("  We'll iterate over all expected entries (public.jsonl). If a generated entry is missing, it's counted as a mismatch.")
    print("="*80 + "\n")

# -------------------------
# Compare all expected entries against generated (by index)
# Use expected's 'type' to assign per-type totals
# -------------------------
stats = defaultdict(lambda: {"total": 0, "mismatch": 0})
mismatch_entries = []
total = len_expected
mismatches = 0

print("\n======================= MISMATCHED ANSWERS =======================\n")

for idx in range(len_expected):
    exp = expected[idx]
    # expected fields
    qtype = exp.get("type", "UNKNOWN")
    paragraph = exp.get("paragraph", "").strip()
    exp_country_raw = exp.get("country", "")    # note: earlier you had "country" key
    exp_city_raw = exp.get("city", "")

    exp_country = normalize(exp_country_raw)
    exp_city = normalize(exp_city_raw)

    # increment total for this type
    stats[qtype]["total"] += 1

    # get generated for this index (if available)
    gen = generated[idx] if idx < len_generated else None

    if gen is None:
        # missing or unparsable generated entry - treat as mismatch
        gen_country_raw = ""
        gen_city_raw = ""
        gen_country = ""
        gen_city = ""
        country_match = False
        city_match = False
        is_mismatch = True
        note = "MISSING_GENERATED"
    else:
        gen_country_raw = gen.get("country", "")
        gen_city_raw = gen.get("city", "")
        gen_country = normalize(gen_country_raw)
        gen_city = normalize(gen_city_raw)
        country_match = (exp_country == gen_country)
        city_match = (exp_city == gen_city)
        # if expected city is empty and generated city is empty -> treat as match for city (but overall rule still requires both to match)
        # Our rule: both must match to be considered MATCH; if either mismatches, entire item is mismatch
        is_mismatch = not (country_match and city_match)
        note = ""

    if is_mismatch:
        stats[qtype]["mismatch"] += 1
        mismatches += 1

        # print mismatch detail (keeps the format you showed)
        print(f"[Question {idx+1}] Type: {qtype}")
        print("-" * 80)
        if paragraph:
            # show a shortened paragraph if it's huge
            preview = paragraph if len(paragraph) < 1000 else (paragraph[:1000] + " ...")
            print(f"Paragraph: {preview}\n")
        print("Expected Answer:")
        print(f"  Country: {exp.get('country','')}")
        print(f"  City: {exp.get('city','')}\n")
        if gen is None:
            print("Generated Answer:  MISSING (no corresponding generated line)")
        else:
            print("Generated Answer:")
            print(f"  Country: {gen.get('country','')}")
            print(f"  City: {gen.get('city','')}")
        print("\nMatch Status:")
        print(f"  Country: {'✓ MATCH' if country_match else '✗ MISMATCH'}")
        print(f"  City:    {'✓ MATCH' if city_match else '✗ MISMATCH'}")
        if note:
            print(f"\nNote: {note}")
        print("\n" + "-" * 80 + "\n")

# -------------------------
# Final statistics printing
# -------------------------
accuracy = 100 * (total - mismatches) / total if total else 0.0

print("\n" + "="*70)
print("OVERALL STATISTICS")
print("="*70)
print(f"Total Questions: {total}")
print(f"Mismatched Answers: {mismatches}")
print(f"Accuracy: {accuracy:.2f}%")

print("\n" + "="*70)
print("STATISTICS BY QUESTION TYPE")
print("="*70)

for qtype, st in sorted(stats.items()):
    t = st["total"]
    m = st["mismatch"]
    acc = 100 * (t - m) / t if t else 0.0
    print(f"\n{qtype}:")
    print(f"  Total Questions: {t}")
    print(f"  Mismatched Answers: {m}")
    print(f"  Accuracy: {acc:.2f}%")



======================= MISMATCHED ANSWERS =======================

[Question 1] Type: GlobalTrekker
--------------------------------------------------------------------------------
Paragraph: Uluru, also known as Ayers Rock, is a large sandstone monolith that is sacred to the Pitjantjatjara, the Aboriginal people of the area, known as the Aṉangu. It is one of the most important indigenous sites of the country, being a popular destination for tourists since the 1930s. The area around the formation is home to many springs, waterholes, rock caves and ancient paintings. Uluru is listed as a UNESCO World Heritage Site.

Expected Answer:
  Country: Australia
  City: 

Generated Answer:
  Country: Australia
  City: Unknown

Match Status:
  Country: ✓ MATCH
  City:    ✗ MISMATCH

--------------------------------------------------------------------------------

[Question 3] Type: GlobalTrekker
--------------------------------------------------------------------------------
Paragraph: Paradise